In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyrender
import os
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.autograd import Variable
from torchsummary import summary
import skimage.measure as sk

import h5py

import time
import pymesh
import trimesh

#%matplotlib notebook

In [3]:
device = torch.device("cuda")

# reproducible.
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.manual_seed(0)

In [4]:
device

device(type='cuda')

In [5]:
class ChairDepthDataset(Dataset):
    
    def __init__(self, h5_file):
        
        self.hf = h5py.File(h5_file, 'r')
        self.keys = list(self.hf.keys())
        
        self.to_tensor = transforms.ToTensor()

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        
        group = self.hf[self.keys[idx]]
        
        model_id = str(self.keys[idx])
        depth_img = self.to_tensor(Image.fromarray(np.array(group['depth_img'])))
        azimuth = float(group['azimuth'][()])
        elevation = float(group['elevation'][()])
        distance = float(group['distance'][()])
        target_vox = torch.tensor(group['target_vox'], dtype=torch.float)
        
        sample = {'model_id': model_id,
                  'depth_img': depth_img,
                  'azimuth': azimuth,
                  'elevation': elevation,
                  'distance': distance,
                  'target_vox': target_vox, 
                 }

        return sample

In [6]:
data_file = '/home/ankbzpx/datasets/ShapeNet/ShapeNetRenderingh5_v1/03001627/data_test_rescale.h5'

test_depth_dataset = ChairDepthDataset(data_file)

In [7]:
# plot vox or img
from vis_utils import plotFromVoxels, plotImg, plot_image_list
# generate with continuous model
from render_utils import generate_mesh, get_relative_transform_matrix
# render mesh with pyrender
from render_utils import render
# mesh from ground truth vox
from render_utils import RotateAlongAxis
# transfrom sdf
from render_utils import get_transformed_indices, sdf2Voxel, get_meshgrid, get_transformed_meshgrid, get_relative_transformed_vox
# get cd, emd, iou from 2 pymesh
from render_utils import get_test_results

In [8]:
# Pre-trained Model path
discrete_encoder_path = 'discrete_encoder.pth'
mapping_path = 'mapping.pth'
discrete_decoder_path = 'discrete_decoder.pth'
unet_path = 'con_unet_full.pth'
continuous_model_path = 'continuous_model.pth'


from models import Discrete_encoder, Mapping, Discrete_decoder, Conditional_UNET, Continuous

####################
# Discrete Encoder #
####################

discrete_encoder = Discrete_encoder().to(device)
discrete_encoder.load_state_dict(torch.load(discrete_encoder_path))
discrete_encoder.eval()

for child in discrete_encoder.children():
    for param in child.parameters():
        param.requires_grad = False


# ###########
# # Mapping #
# ###########


mapping = Mapping().to(device)
mapping.load_state_dict(torch.load(mapping_path))
mapping.eval()

for child in mapping.children():
    for param in child.parameters():
        param.requires_grad = False

# ####################
# # Discrete Decoder #
# ####################

discrete_decoder = Discrete_decoder().to(device)
discrete_decoder.load_state_dict(torch.load(discrete_decoder_path))
discrete_decoder.eval()
########
# UNET #
########

# pre-trained model is loaded within the model
unet = Conditional_UNET(unet_path).to(device)

unet.eval()

for child in unet.children():
    for param in child.parameters():
        param.requires_grad = False


        
##############
# Continuous #
##############

continuous = Continuous().to(device)
continuous.load_state_dict(torch.load(continuous_model_path))
continuous.eval()

for child in continuous.children():
    for param in child.parameters():
        param.requires_grad = False

In [9]:
voxsize = 32
D2R = np.pi/180.0

In [10]:
# custom depth consistency loss

from torch.autograd import Function
from torch.autograd.function import once_differentiable

class DepthConsistencyLoss(Function):

    @staticmethod
    def forward(ctx, out_vox, depth_index, close_index):
        
        # out_vox shape n x 1 x 32 x 32 x 32
        # depth_index shape n x 32 x 32 x 32
        # close_index shape n x 32 x 32 x 32
        
        ctx.save_for_backward(out_vox)
        
        vox = out_vox.squeeze(1)
        ctx.depth_index = depth_index
        ctx.close_index = close_index
        
        loss = torch.zeros_like(depth_index, dtype=torch.float).to(device)
        
        #bce form
        loss[depth_index.bool()] = -torch.log(vox[depth_index.bool()])
        loss[close_index.bool()] = -torch.log(1-vox[close_index.bool()])
        
        return torch.sum(loss) / (torch.sum(depth_index) + torch.sum(close_index))
        
    @staticmethod
    @once_differentiable
    def backward(ctx, grad_output):
        
        out_vox, = ctx.saved_tensors
        
        vox = out_vox.squeeze(1)
        grad = torch.zeros_like(ctx.depth_index, dtype=torch.float).to(device)
        
        #bce form
        grad[ctx.depth_index.bool()] = -1/vox[ctx.depth_index.bool()]
        grad[ctx.close_index.bool()] = 1/(1-vox[ctx.close_index.bool()])
        
        return grad.unsqueeze(1), None, None, None

In [11]:
from refine_utils import get_radius, get_depth_close_idx

In [12]:
from render_utils import mesh_from_voxel

In [13]:
%matplotlib notebook

mesh_grid_res = 64
mesh_batch_size = 128

def depth_refine_three_imgs(idx, lr = 1e-3, iter_count = 30):
    
    data_from = test_depth_dataset[24*idx]
    depth_img_from = data_from['depth_img'].unsqueeze(0)
    target_vox_from = data_from['target_vox']
    distance_from = data_from['distance']
    azimuth_from = data_from['azimuth']
    elevation_from = data_from['elevation']
    
    #plotFromVoxels(target_vox_from[0])
    
    ########### initial depth ##########
    
    radius = get_radius(depth_img_from, distance_from)
    depth_index_from, close_index_from = get_depth_close_idx(depth_img_from, distance_from, radius)
    
    ########### gt mesh ##########
    
    model_dir = path_model + test_depth_dataset[24*idx]['model_id'].split('_')[0] + '/model.obj'
    mesh_py = pymesh.load_mesh(model_dir)
    transformed_vertices = get_transformed_indices(mesh_py.vertices, azimuth_from, elevation_from, 1)
    gt_radius = np.max(np.linalg.norm(transformed_vertices, axis = 1))
    mesh_gt = pymesh.form_mesh(transformed_vertices/gt_radius, mesh_py.faces)
    
    #plotImg(depth_img_from[0, 0].detach().cpu().numpy())
    
    z = discrete_encoder(depth_img_from.to(device))
    w = mapping(z)
    
    w = w.clone().detach().requires_grad_(True)
    
    w.requires_grad = True
    
    optimizer = torch.optim.SGD([w], lr=lr, momentum=0.8)
    
    out_vox = torch.sigmoid(discrete_decoder(w))
    #plotFromVoxels(out_vox[0, 0].detach().cpu().numpy() > 0.5)
    
    mesh_vox_before = mesh_from_voxel(out_vox[0, 0])
    
    mesh_before = generate_mesh(continuous, unet, out_vox, z, device, vox_res = 32, grid_res = mesh_grid_res, batch_size = mesh_batch_size, 
                                azimuth = 0, elevation = 0, isosurface = 0.0)
    
    ran = np.random.choice(np.arange(1, 24), 2)
    
    ########### first depth ##########
    
    idx_to_1 = 24*idx + ran[0]
    data_to_1 = test_depth_dataset[idx_to_1]
    depth_img_to_1 = data_to_1['depth_img'].unsqueeze(0)
    distance_to_1 = data_to_1['distance']
    azimuth_to_1 = data_to_1['azimuth']
    elevation_to_1 = data_to_1['elevation']
    depth_index_to_1, close_index_to_1 = get_depth_close_idx(depth_img_to_1, distance_to_1, radius)
    
    ########### second depth ##########
    
    idx_to_2 = 24*idx + ran[1]
    data_to_2 = test_depth_dataset[idx_to_2]
    depth_img_to_2 = data_to_2['depth_img'].unsqueeze(0)
    distance_to_2 = data_to_2['distance']
    azimuth_to_2 = data_to_2['azimuth']
    elevation_to_2 = data_to_2['elevation']
    depth_index_to_2, close_index_to_2 = get_depth_close_idx(depth_img_to_2, distance_to_2, radius)
    
    # Optimize with depth consistency loss
    for i in range(iter_count):
        
        optimizer.zero_grad()
        
        
        relative_transformed_vox_1 = get_relative_transformed_vox(out_vox, -azimuth_from, -elevation_from, azimuth_to_1, elevation_to_1, 
                                                            device, voxsize = 32, align_mode = 'zeros')
        
        relative_transformed_vox_2 = get_relative_transformed_vox(out_vox, -azimuth_from, -elevation_from, azimuth_to_2, elevation_to_2, 
                                                            device, voxsize = 32, align_mode = 'zeros')
        
        loss_0 = DepthConsistencyLoss.apply(out_vox, depth_index_from.unsqueeze(0).to(device), close_index_from.unsqueeze(0).to(device))
        loss_1 = DepthConsistencyLoss.apply(relative_transformed_vox_1, depth_index_to_1.unsqueeze(0).to(device), close_index_to_1.unsqueeze(0).to(device))
        loss_2 = DepthConsistencyLoss.apply(relative_transformed_vox_2, depth_index_to_2.unsqueeze(0).to(device), close_index_to_2.unsqueeze(0).to(device))
        
        loss = 2*loss_0 + loss_1 + loss_2
        
#         if i == 0:
#             print("Initial loss: ", loss.item())
#         else:
#             print(loss.item())
        
        loss.backward(retain_graph=True)
        optimizer.step()
        
        out_vox = torch.sigmoid(discrete_decoder(w))
        
    #plotFromVoxels(out_vox[0, 0].detach().cpu().numpy() > 0.5)    
    
    mesh_after = generate_mesh(continuous, unet, out_vox, z, device, vox_res = 32, grid_res = mesh_grid_res, batch_size = mesh_batch_size, 
                               azimuth = 0, elevation = 0, isosurface = 0.0)
    
    mesh_vox_after = mesh_from_voxel(out_vox[0, 0])
    
    return mesh_gt, mesh_vox_before, mesh_vox_after, mesh_before, mesh_after

In [14]:
def normalize_mesh(mesh_py, vox_size = 64):
    
    normalized_vertices = mesh_py.vertices - (vox_size/2)
    normalized_vertices /= np.max(np.linalg.norm(normalized_vertices, axis = 1))
    
    return pymesh.form_mesh(normalized_vertices, mesh_py.faces)

In [15]:
path_model = '/home/ankbzpx/datasets/ShapeNet/ShapeNetCore.v1/03001627/'

cd_gt_v_b_list = []
emd_gt_v_b_list = []
iou_gt_v_b_list = []

cd_gt_v_a_list = []
emd_gt_v_a_list = []
iou_gt_v_a_list = []

cd_gt_m_b_list = []
emd_gt_m_b_list = []
iou_gt_m_b_list = []

cd_gt_m_a_list = []
emd_gt_m_a_list = []
iou_gt_m_a_list = []

failed_idx = []

for idx in range(int(len(test_depth_dataset)/24)):
    print(idx)
    start_time = time.time()
    
    mesh_gt, mesh_vox_before, mesh_vox_after, mesh_before, mesh_after = depth_refine_three_imgs(idx)
    
    if mesh_gt is None or mesh_vox_before is None or mesh_vox_after is None or mesh_before is None or mesh_after is None:
        print('Failed case')
        failed_idx.append(idx)
        continue
        
    #mesh_gt = normalize_mesh(mesh_gt)
    mesh_vox_before = normalize_mesh(mesh_vox_before, 64)
    mesh_vox_after = normalize_mesh(mesh_vox_after, 64)
    mesh_before = normalize_mesh(mesh_before, mesh_grid_res)
    mesh_after = normalize_mesh(mesh_after, mesh_grid_res)
    
    cd_gt_v_b, emd_gt_v_b, iou_gt_v_b = get_test_results(mesh_gt, mesh_vox_before)
    cd_gt_v_a, emd_gt_v_a, iou_gt_v_a = get_test_results(mesh_gt, mesh_vox_after)
    cd_gt_m_b, emd_gt_m_b, iou_gt_m_b = get_test_results(mesh_gt, mesh_before)
    cd_gt_m_a, emd_gt_m_a, iou_gt_m_a = get_test_results(mesh_gt, mesh_after)
    
    cd_gt_v_b_list.append(cd_gt_v_b)
    emd_gt_v_b_list.append(emd_gt_v_b)
    iou_gt_v_b_list.append(iou_gt_v_b)
    
    cd_gt_v_a_list.append(cd_gt_v_a)
    emd_gt_v_a_list.append(emd_gt_v_a)
    iou_gt_v_a_list.append(iou_gt_v_a)
    
    cd_gt_m_b_list.append(cd_gt_m_b)
    emd_gt_m_b_list.append(emd_gt_m_b)
    iou_gt_m_b_list.append(iou_gt_m_b)
    
    cd_gt_m_a_list.append(cd_gt_m_a)
    emd_gt_m_a_list.append(emd_gt_m_a)
    iou_gt_m_a_list.append(iou_gt_m_a)
    
    print("--- %s seconds ---" % (time.time() - start_time))

0
--- 7.335648775100708 seconds ---
1
--- 6.120589971542358 seconds ---
2
--- 6.29094123840332 seconds ---
3
--- 7.43862509727478 seconds ---
4
--- 5.909606218338013 seconds ---
5
--- 5.745596408843994 seconds ---
6
--- 5.591710090637207 seconds ---
7
--- 5.633188486099243 seconds ---
8
--- 5.132876873016357 seconds ---
9
--- 6.042597532272339 seconds ---
10
--- 8.119388103485107 seconds ---
11
--- 6.183493614196777 seconds ---
12
--- 6.541289806365967 seconds ---
13
--- 5.782090187072754 seconds ---
14
--- 5.509189605712891 seconds ---
15
--- 5.021416902542114 seconds ---
16
--- 5.957952976226807 seconds ---
17
--- 7.19655442237854 seconds ---
18
--- 5.639666795730591 seconds ---
19
--- 5.354566812515259 seconds ---
20
--- 6.967867136001587 seconds ---
21
--- 5.524966478347778 seconds ---
22
--- 5.5699052810668945 seconds ---
23
--- 5.652772903442383 seconds ---
24
--- 5.82321572303772 seconds ---
25
--- 5.4928483963012695 seconds ---
26
--- 6.0208680629730225 seconds ---
27
--- 5.301

--- 8.405008554458618 seconds ---
219
--- 5.809110403060913 seconds ---
220
--- 5.571713209152222 seconds ---
221
--- 5.3865156173706055 seconds ---
222
--- 6.899624586105347 seconds ---
223
--- 5.937293291091919 seconds ---
224
--- 5.642299652099609 seconds ---
225
--- 5.470247268676758 seconds ---
226
--- 6.787536144256592 seconds ---
227
--- 5.581496477127075 seconds ---
228
--- 5.534604549407959 seconds ---
229
--- 5.61357045173645 seconds ---
230
--- 5.878158330917358 seconds ---
231
--- 5.327233552932739 seconds ---
232
--- 6.075501918792725 seconds ---
233
--- 5.407459497451782 seconds ---
234
--- 5.921319007873535 seconds ---
235
--- 5.570752382278442 seconds ---
236
--- 5.642833471298218 seconds ---
237
--- 5.44344425201416 seconds ---
238
--- 5.591467380523682 seconds ---
239
--- 5.438242197036743 seconds ---
240
--- 5.620993614196777 seconds ---
241
--- 5.573693037033081 seconds ---
242
--- 5.378359794616699 seconds ---
243
--- 5.719496726989746 seconds ---
244
--- 5.5791304

--- 5.729416608810425 seconds ---
435
--- 5.699968099594116 seconds ---
436
--- 6.038259029388428 seconds ---
437
--- 7.783782958984375 seconds ---
438
--- 7.348893642425537 seconds ---
439
--- 5.460253715515137 seconds ---
440
--- 6.339035987854004 seconds ---
441
--- 8.889048337936401 seconds ---
442
--- 5.673578262329102 seconds ---
443
--- 5.8870744705200195 seconds ---
444
--- 6.346792459487915 seconds ---
445
--- 7.132389307022095 seconds ---
446
--- 6.325475692749023 seconds ---
447
--- 6.923911809921265 seconds ---
448
--- 7.818133354187012 seconds ---
449
--- 5.81351637840271 seconds ---
450
--- 5.373084545135498 seconds ---
451
--- 7.178707838058472 seconds ---
452
--- 5.793130159378052 seconds ---
453
--- 6.538401126861572 seconds ---
454
--- 6.195241451263428 seconds ---
455
--- 7.011888742446899 seconds ---
456
--- 5.668123960494995 seconds ---
457
--- 7.624004125595093 seconds ---
458
--- 5.358569622039795 seconds ---
459
--- 5.7391626834869385 seconds ---
460
--- 6.30952

--- 6.6839210987091064 seconds ---
651
--- 5.673067569732666 seconds ---
652
--- 5.914558172225952 seconds ---
653
--- 5.583383083343506 seconds ---
654
--- 5.550501346588135 seconds ---
655
--- 6.130421876907349 seconds ---
656
--- 6.017101287841797 seconds ---
657
--- 5.669975280761719 seconds ---
658
--- 7.19110631942749 seconds ---
659
--- 5.974357843399048 seconds ---
660
--- 6.869552373886108 seconds ---
661
--- 8.484474420547485 seconds ---
662
--- 7.775155544281006 seconds ---
663
--- 6.320711612701416 seconds ---
664
--- 6.025375604629517 seconds ---
665
--- 5.516026258468628 seconds ---
666
--- 5.372476816177368 seconds ---
667
--- 5.564741373062134 seconds ---
668
--- 6.301836729049683 seconds ---
669
--- 5.482944488525391 seconds ---
670
--- 6.33527684211731 seconds ---
671
--- 5.899300813674927 seconds ---
672
--- 5.5648767948150635 seconds ---
673
--- 6.082932472229004 seconds ---
674
--- 5.807833194732666 seconds ---
675
--- 6.18155312538147 seconds ---
676
--- 5.6340937

--- 6.113160848617554 seconds ---
867
--- 5.838565349578857 seconds ---
868
--- 5.994654417037964 seconds ---
869
--- 6.445880651473999 seconds ---
870
--- 7.984138250350952 seconds ---
871
--- 6.659804582595825 seconds ---
872
--- 5.559663534164429 seconds ---
873
--- 5.543359279632568 seconds ---
874
--- 5.833638429641724 seconds ---
875
--- 7.444502592086792 seconds ---
876
--- 8.149993419647217 seconds ---
877
--- 5.685385227203369 seconds ---
878
--- 5.601856470108032 seconds ---
879
--- 6.146292448043823 seconds ---
880
--- 6.302429676055908 seconds ---
881
--- 5.62645697593689 seconds ---
882
--- 6.036393165588379 seconds ---
883
--- 7.039531230926514 seconds ---
884
--- 6.457382917404175 seconds ---
885
--- 7.795533180236816 seconds ---
886
--- 5.516827583312988 seconds ---
887
--- 5.337373971939087 seconds ---
888
--- 5.421212673187256 seconds ---
889
--- 5.957656621932983 seconds ---
890
--- 6.6719324588775635 seconds ---
891
--- 6.553710460662842 seconds ---
892
--- 6.164549

--- 5.386014699935913 seconds ---
1081
--- 5.579824209213257 seconds ---
1082
--- 5.684067726135254 seconds ---
1083
--- 7.539417028427124 seconds ---
1084
--- 5.491586446762085 seconds ---
1085
--- 7.052948236465454 seconds ---
1086
--- 5.485695123672485 seconds ---
1087
--- 5.611055850982666 seconds ---
1088
--- 5.747910976409912 seconds ---
1089
--- 6.178895473480225 seconds ---
1090
--- 5.684150695800781 seconds ---
1091
--- 8.313200235366821 seconds ---
1092
--- 5.652819871902466 seconds ---
1093
--- 5.386085271835327 seconds ---
1094
--- 5.722363233566284 seconds ---
1095
--- 6.361860036849976 seconds ---
1096
--- 7.030585765838623 seconds ---
1097
--- 6.200638055801392 seconds ---
1098
--- 6.089748382568359 seconds ---
1099
--- 6.030660390853882 seconds ---
1100
--- 6.939401149749756 seconds ---
1101
--- 6.335933208465576 seconds ---
1102
--- 5.591634750366211 seconds ---
1103
--- 5.416074514389038 seconds ---
1104
--- 5.501314401626587 seconds ---
1105
--- 7.546546220779419 sec

--- 6.304394721984863 seconds ---
1292
--- 7.515067100524902 seconds ---
1293
--- 6.0616655349731445 seconds ---
1294
--- 5.565199613571167 seconds ---
1295
--- 5.662966728210449 seconds ---
1296
--- 5.535473585128784 seconds ---
1297
--- 6.245340347290039 seconds ---
1298
--- 5.776651382446289 seconds ---
1299
--- 7.443344593048096 seconds ---
1300
--- 8.419670581817627 seconds ---
1301
--- 6.198106288909912 seconds ---
1302
--- 5.707306146621704 seconds ---
1303
--- 5.979755163192749 seconds ---
1304
--- 7.150829553604126 seconds ---
1305
--- 6.088781118392944 seconds ---
1306
--- 5.500072717666626 seconds ---
1307
--- 5.607320308685303 seconds ---
1308
--- 6.140923023223877 seconds ---
1309
--- 6.053291320800781 seconds ---
1310
--- 5.495731830596924 seconds ---


In [16]:
print('--- vox before ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_v_b_list), ' s_', np.std(cd_gt_v_b_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_v_b_list), ' s_', np.std(emd_gt_v_b_list))
print()
print('Intersection over Union: m_', np.mean(iou_gt_v_b_list), ' s_', np.std(iou_gt_v_b_list))
print()

print('--- vox after ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_v_a_list), ' s_', np.std(cd_gt_v_a_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_v_a_list), ' s_', np.std(emd_gt_v_a_list))
print()
print('Intersection over Union: m_', np.mean(iou_gt_v_a_list), ' s_', np.std(iou_gt_v_a_list))
print()

print('--- mesh before ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_m_b_list), ' s_', np.std(cd_gt_m_b_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_m_b_list), ' s_', np.std(emd_gt_m_b_list))
print()
print('Intersection over Union: m_', np.mean(iou_gt_m_b_list), ' s_', np.std(iou_gt_m_b_list))
print()

print('--- mesh after ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_m_a_list), ' s_', np.std(cd_gt_m_a_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_m_a_list), ' s_', np.std(emd_gt_m_a_list))
print()
print('Intersection over Union m_', np.mean(iou_gt_m_a_list), ' s_', np.std(iou_gt_m_a_list))
print()

--- vox before ---
Chamfer Distacne: m_ 0.022222808  s_ 0.008482305

Earth Movers Distance: m_ 0.04006497164880166  s_ 0.016278286333165148

Intersection over Union: m_ 0.5427513236414269  s_ 0.12252359841889883

--- vox after ---
Chamfer Distacne: m_ 0.02376638  s_ 0.008424185

Earth Movers Distance: m_ 0.044680847418037226  s_ 0.01734986132308128

Intersection over Union: m_ 0.5351377005674767  s_ 0.12006001753699867

--- mesh before ---
Chamfer Distacne: m_ 0.022635398  s_ 0.015557701

Earth Movers Distance: m_ 0.028833799993615458  s_ 0.020837837624605414

Intersection over Union: m_ 0.5905169058294869  s_ 0.14430601871687798

--- mesh after ---
Chamfer Distacne: m_ 0.018825116  s_ 0.008653481

Earth Movers Distance: m_ 0.028689540134476388  s_ 0.016883975057434674

Intersection over Union m_ 0.6026711811583283  s_ 0.12372119211082773



In [ ]:
render(mesh_gt)

In [34]:
render(mesh_vox_before)

In [35]:
render(mesh_vox_after)

In [17]:
render(mesh_before)

In [37]:
render(mesh_after)

In [55]:
int(len(test_depth_dataset)/24)

1311